In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.cm as cm
from matplotlib.ticker import PercentFormatter
from matplotlib.colors import Normalize

import re

In [2]:
# 폰트 설정
import matplotlib as mpl
import matplotlib.font_manager as fm

font_path = r"C:\Windows\Fonts\malgun.ttf"  
font_prop = fm.FontProperties(fname=font_path)
mpl.rcParams['font.family'] = font_prop.get_name()
mpl.rcParams['axes.unicode_minus'] = False

In [3]:
# data load
df = pd.read_csv(r'C:\Users\user\Desktop\연구\5. 국방부 용역과제\250730.csv')
df_geo = pd.read_csv(r'..\datasets\df_geo_new.csv') # 위경도 좌표

print(df.head())
print(df_geo.head())

   순번                   출발지 주소              중간집결지 주소            최종 도착지 주소  \
0   1  경기도 고양시 덕양구 중앙로633번길 12   경기도 고양시 덕양구 혜음로 142  경기도 고양시 덕양구 혜음로 142   
1   2   대구광역시 서구 북비산로61길 20-24      경상북도 경주시 태종로 516  경기도 고양시 덕양구 동헌로 305   
2   3   서울특별시 양천구 목동중앙본로11길 19  서울특별시 서초구 사평대로45길 26  강원특별자치도 동해시 대동로 430   
3   4        서울특별시 강남구 학동로 432       경기도 과천시 중앙로 294      경기도 과천시 중앙로 294   
4   5         대구광역시 남구 명덕로 236      대구광역시 수성구 무열로 56     대구광역시 수성구 무열로 56   

    중간집결 기한    최종도착 기한 이동방법  
0  0+1일 14시   0+1일 14시   개별  
1  0+28일 9시  0+28일 14시   집단  
2   0+3일 8시   0+3일 14시   집단  
3  0+1일 14시   0+1일 14시   개별  
4  0+3일 14시   0+3일 14시   개별  
   num  start_lat   start_lng    mid_lat     mid_lng    end_lat     end_lng
0    1  37.626949  126.827581  37.713020  126.902443  37.713020  126.902443
1    2  35.881731  128.567540  35.839566  129.185196  37.713651  126.889915
2    3  37.540484  126.867756  37.506451  127.012692  37.503393  129.130509
3    4  37.517981  127.048764  37.441493  126.996973 

In [89]:
# Merge
df_merge = pd.merge(df, df_geo, left_on = '순번', right_on = 'num', how = 'left').drop(columns = ['num'])

print(df_merge.head(3))
print(df_merge.tail(3))

   순번                   출발지 주소              중간집결지 주소            최종 도착지 주소  \
0   1  경기도 고양시 덕양구 중앙로633번길 12   경기도 고양시 덕양구 혜음로 142  경기도 고양시 덕양구 혜음로 142   
1   2   대구광역시 서구 북비산로61길 20-24      경상북도 경주시 태종로 516  경기도 고양시 덕양구 동헌로 305   
2   3   서울특별시 양천구 목동중앙본로11길 19  서울특별시 서초구 사평대로45길 26  강원특별자치도 동해시 대동로 430   

    중간집결 기한    최종도착 기한 이동방법  start_lat   start_lng    mid_lat     mid_lng  \
0  0+1일 14시   0+1일 14시   개별  37.626949  126.827581  37.713020  126.902443   
1  0+28일 9시  0+28일 14시   집단  35.881731  128.567540  35.839566  129.185196   
2   0+3일 8시   0+3일 14시   집단  37.540484  126.867756  37.506451  127.012692   

     end_lat     end_lng  
0  37.713020  126.902443  
1  37.713651  126.889915  
2  37.503393  129.130509  
            순번                    출발지 주소               중간집결지 주소  \
815772  815773      광주광역시 광산구 장신로19번길 10       광주광역시 북구 우치로 339   
815773  815774        서울특별시 동작구 사당로8길 55  서울특별시 강동구 양재대로156길 11   
815774  815775  경기도 성남시 중원구 자혜로17번길 42-3     강원특별자치도 원주시 연세대길 1   

     

In [90]:
df_merge = df_merge.dropna().reset_index(drop = True)

In [91]:
# statistics
print(df_merge.shape)
print(df_merge['이동방법'].value_counts())

# 위경도 NA 확인
num_na = df_merge.isna().any(axis = 1).sum()
mask = df_merge.notna().all(axis = 1)
print('num_na:', num_na)

df_merge = df_merge[mask].reset_index(drop = True)
print(df_merge)
print(df_merge['이동방법'].value_counts())
# 변환안된 것: 5,067행
# 사용가능 총: 810,708행

# # 저장
# df_merge.to_csv('bottle.csv', index = False, encoding = 'cp949')

(810708, 13)
이동방법
개별    472214
집단    338494
Name: count, dtype: int64
num_na: 0
            순번                    출발지 주소                  중간집결지 주소  \
0            1   경기도 고양시 덕양구 중앙로633번길 12       경기도 고양시 덕양구 혜음로 142   
1            2    대구광역시 서구 북비산로61길 20-24          경상북도 경주시 태종로 516   
2            3    서울특별시 양천구 목동중앙본로11길 19      서울특별시 서초구 사평대로45길 26   
3            4         서울특별시 강남구 학동로 432           경기도 과천시 중앙로 294   
4            5          대구광역시 남구 명덕로 236          대구광역시 수성구 무열로 56   
...        ...                       ...                       ...   
810703  815771         서울특별시 마포구 만리재로 60          경기도 남양주시 덕릉로 967   
810704  815772           충청남도 아산시 실옥로 15  충청남도 천안시 동남구 천안대로 429-13   
810705  815773      광주광역시 광산구 장신로19번길 10          광주광역시 북구 우치로 339   
810706  815774        서울특별시 동작구 사당로8길 55     서울특별시 강동구 양재대로156길 11   
810707  815775  경기도 성남시 중원구 자혜로17번길 42-3        강원특별자치도 원주시 연세대길 1   

                      최종 도착지 주소   중간집결 기한    최종도착 기한 이동방법  start_lat  \
0      

In [92]:
# 시간 변환
# 중간시간 변환
df_merge_mid = df_merge['중간집결 기한'].str.split('일', expand = True)
df_merge_mid.columns = ['Mday_mid', 'Mhour_mid']
df_merge_mid['Mday_mid'] = df_merge_mid['Mday_mid'].str.replace('0+', '').astype(int)
df_merge_mid['Mhour_mid'] = df_merge_mid['Mhour_mid'].str.replace('시', '').astype(int)
df_merge_mid['mid_td'] = (pd.to_timedelta(df_merge_mid['Mday_mid'], unit = 'D') + pd.to_timedelta(df_merge_mid['Mhour_mid'], unit = 'h'))

# 최종시간 변환
df_merge_end = df_merge['최종도착 기한'].str.split('일', expand = True)
df_merge_end.columns = ['Mday_end', 'Mhour_end']
df_merge_end['Mday_end'] = df_merge_end['Mday_end'].str.replace('0+', '').astype(int)
df_merge_end['Mhour_end'] = df_merge_end['Mhour_end'].str.replace('시', '').astype(int)
df_merge_end['end_td'] = (pd.to_timedelta(df_merge_end['Mday_end'], unit = 'D') + pd.to_timedelta(df_merge_end['Mhour_end'], unit = 'h'))

df_merge = pd.concat([df_merge, df_merge_mid, df_merge_end], axis = 1).drop(columns = ['중간집결 기한', '최종도착 기한', 'Mday_mid', 'Mhour_mid', 'Mday_end', 'Mhour_end']).reset_index(drop = True)
print(df_merge.head(3))

   순번                   출발지 주소              중간집결지 주소            최종 도착지 주소  \
0   1  경기도 고양시 덕양구 중앙로633번길 12   경기도 고양시 덕양구 혜음로 142  경기도 고양시 덕양구 혜음로 142   
1   2   대구광역시 서구 북비산로61길 20-24      경상북도 경주시 태종로 516  경기도 고양시 덕양구 동헌로 305   
2   3   서울특별시 양천구 목동중앙본로11길 19  서울특별시 서초구 사평대로45길 26  강원특별자치도 동해시 대동로 430   

  이동방법  start_lat   start_lng    mid_lat     mid_lng    end_lat     end_lng  \
0   개별  37.626949  126.827581  37.713020  126.902443  37.713020  126.902443   
1   집단  35.881731  128.567540  35.839566  129.185196  37.713651  126.889915   
2   집단  37.540484  126.867756  37.506451  127.012692  37.503393  129.130509   

            mid_td           end_td  
0  1 days 14:00:00  1 days 14:00:00  
1 28 days 09:00:00 28 days 14:00:00  
2  3 days 08:00:00  3 days 14:00:00  


In [93]:
# M+7일 이내 움직이는 사람만
df_merge = df_merge[df_merge['mid_td'].dt.days <= 7]

In [94]:
# pattern = r'^(.*?(?:도|특별시|광역시|특별자치도)\s*[^\s]+?(?:시|군|구))'

def extract_province_city(addr):
    if addr.startswith('서울특별시'):
        m = re.match('^(서울특별시\s*[^\s]+구)', addr)
        return m.group(1) if m else '서울특별시'
    m = re.match(r'^(.*?(?:도|광역시|특별자치도|특별자치시))', addr)
    return m.group(1) if m else 'None'
    
df_merge["도시단위"] = df_merge["출발지 주소"].apply(extract_province_city)
print(df_merge)

<>:5: SyntaxWarning: invalid escape sequence '\s'
<>:5: SyntaxWarning: invalid escape sequence '\s'
C:\Users\user\AppData\Local\Temp\ipykernel_27564\1493526492.py:5: SyntaxWarning: invalid escape sequence '\s'
  m = re.match('^(서울특별시\s*[^\s]+구)', addr)


            순번                    출발지 주소                   중간집결지 주소  \
0            1   경기도 고양시 덕양구 중앙로633번길 12        경기도 고양시 덕양구 혜음로 142   
2            3    서울특별시 양천구 목동중앙본로11길 19       서울특별시 서초구 사평대로45길 26   
3            4         서울특별시 강남구 학동로 432            경기도 과천시 중앙로 294   
4            5          대구광역시 남구 명덕로 236           대구광역시 수성구 무열로 56   
6            7    경상남도 창원시 성산구 외동반림로 219  경상남도 창원시 성산구 중앙대로250번길 13   
...        ...                       ...                        ...   
810703  815771         서울특별시 마포구 만리재로 60           경기도 남양주시 덕릉로 967   
810704  815772           충청남도 아산시 실옥로 15   충청남도 천안시 동남구 천안대로 429-13   
810705  815773      광주광역시 광산구 장신로19번길 10           광주광역시 북구 우치로 339   
810706  815774        서울특별시 동작구 사당로8길 55      서울특별시 강동구 양재대로156길 11   
810707  815775  경기도 성남시 중원구 자혜로17번길 42-3         강원특별자치도 원주시 연세대길 1   

                        최종 도착지 주소 이동방법  start_lat   start_lng    mid_lat  \
0             경기도 고양시 덕양구 혜음로 142   개별  37.626949  126.827581  37.71302

In [32]:
from geopy.geocoders import Nominatim

geolocator = Nominatim(user_agent="geo_korea")
lat = []
lng = []
city = list(df_merge['도시단위'].unique())
for idx in city:
    location = geolocator.geocode(idx)
    lat.append(location.latitude)
    lng.append(location.longitude)

    print((location.latitude, location.longitude))


(37.2884675, 127.0535231)
(35.8713, 128.6018)
(37.5171, 126.8663)
(37.5177, 127.0473)
(36.6357, 127.4915)
(35.2382, 128.6925)
(37.885, 127.7297)
(37.456, 126.7052)
(36.5754, 128.5058)
(35.1799528, 129.0752365)
(34.8159, 126.4629)
(37.4782, 126.9518)
(36.6593, 126.6729)
(37.53, 127.1237)
(37.6686, 127.0466)
(37.5120999, 126.9395)
(37.6063, 127.093)
(37.5362696, 127.0879515)
(37.4951999, 126.8877)
(35.1594647, 126.8515034)
(36.3497007, 127.3849016)
(37.6024, 126.9293)
(37.5790747, 126.9367861)
(37.4835, 127.0322)
(35.8198999, 127.109)
(35.5391697, 129.3119136)
(37.5509, 126.8497)
(37.5262, 126.8959)
(37.59, 127.0165)
(36.4799999, 127.289)
(37.566571, 126.9015317)
(37.5635, 127.0365)
(37.5806949, 126.9827989)
(37.5145, 127.1058)
(37.6395, 127.0255)
(37.5636559, 126.9975097)
(37.654, 127.0567)
(37.4565, 126.8954)
(37.5741982, 127.0395092)
(37.5323, 126.99)
(33.4887737, 126.4987083)
(37.885, 127.7297)


In [95]:
df_solo = df_merge[df_merge['이동방법'] == '개별']
df_grp = df_merge[df_merge['이동방법'] == '집단']

In [97]:
df_solo.head(3)

,순번,출발지 주소,중간집결지 주소,최종 도착지 주소,이동방법,start_lat,start_lng,mid_lat,mid_lng,end_lat,end_lng,mid_td,end_td,도시단위
0,1,경기도 고양시 덕양구 중앙로633번길 12,경기도 고양시 덕양구 혜음로 142,경기도 고양시 덕양구 혜음로 142,개별,37.626949,126.827581,37.713020,126.902443,37.713020,126.902443,1 days 14:00:00,1 days 14:00:00,경기도
3,4,서울특별시 강남구 학동로 432,경기도 과천시 중앙로 294,경기도 과천시 중앙로 294,개별,37.517981,127.048764,37.441493,126.996973,37.441493,126.996973,1 days 14:00:00,1 days 14:00:00,서울특별시 강남구
4,5,대구광역시 남구 명덕로 236,대구광역시 수성구 무열로 56,대구광역시 수성구 무열로 56,개별,35.854689,128.595157,35.863198,128.651230,35.863198,128.651230,3 days 14:00:00,3 days 14:00:00,대구광역시


In [98]:
# 개인 - 출발
from sklearn.preprocessing import MinMaxScaler
counts = [df_solo[df_solo['도시단위'] == idx].shape[0] for idx in city]

import folium

data = pd.DataFrame({
    'city': city,
    'lat': lat,
    'lon': lng,
    'value': counts
})
scaler = MinMaxScaler((5, 30))  # 최소 5, 최대 30 픽셀
data['scaled'] = scaler.fit_transform(data[['value']])
m = folium.Map(location=[36.5, 127.8], zoom_start=7, tiles='OpenStreetMap')

for i, row in data.iterrows():
    folium.CircleMarker(
        location=[row['lat'], row['lon']],
        radius= row['scaled'] ,   # 값에 비례한 버블 크기
        color='crimson',
        fill=True,
        fill_color='crimson',
        fill_opacity=0.6,
        popup=f"{row['city']} ({row['value']})"
    ).add_to(m)

m.save("korea_bubble_map.html")
m


In [99]:
# 그룹 - 출발
from sklearn.preprocessing import MinMaxScaler
counts = [df_grp[df_grp['도시단위'] == idx].shape[0] for idx in city]

import folium

data = pd.DataFrame({
    'city': city,
    'lat': lat,
    'lon': lng,
    'value': counts
})
scaler = MinMaxScaler((5, 30))  # 최소 5, 최대 30 픽셀
data['scaled'] = scaler.fit_transform(data[['value']])
m = folium.Map(location=[36.5, 127.8], zoom_start=7, tiles='OpenStreetMap')

for i, row in data.iterrows():
    folium.CircleMarker(
        location=[row['lat'], row['lon']],
        radius= row['scaled'] ,   # 값에 비례한 버블 크기
        color='crimson',
        fill=True,
        fill_color='crimson',
        fill_opacity=0.6,
        popup=f"{row['city']} ({row['value']})"
    ).add_to(m)

m.save("korea_bubble_map.html")
m
